# Localizing mouse paws from each frame with a fine-tunned Yolo Model 
Author : Ramin Anushiravani

Date: January 29/22 

In [5]:
import numpy as np
import matplotlib.pylab as plt
import os
from PIL import Image
import time
from IPython.display import clear_output
import pickle
from scipy.signal import argrelextrema
from scipy import signal
import pandas as pd
from scipy.fft import fft, ifft
import sys
sys.path.append('../scripts/')
import inference_utils
from yolo import *


In [3]:
# Instantiating a Yolo object for inference 
yoloObj = YOLO(model_path = "../input/trained-weights/trained_weights_final_mouse.h5",
     classes_path = '../input/model-data/model_data/mouse_classes.txt' ,
     anchors_path = '../input/model-data/model_data/yolo_anchors_mouse.txt',
     model_image_size = (64,256), gpu_num = 0, score = 0.3, iou = 0.4)

In [4]:
# Collecting all frame filenames from a video 
video_name = "039350_left_left_20cms_0degUP"
image_path = "../input/mouse-on-treadmill-frames/frames/" + video_name + "/frames/"
files = sorted(os.listdir(image_path), key = lambda x : int(x.split('.')[0]))
files_abs_path = [image_path+f for f in files ]

In [10]:
final_walking_frames = split_video_to_walking_only(files_abs_path)

### Performing Paw Detection Inference

In [14]:
dict_paws = paw_detection_video(files_abs_path)

### Finding the average number of steps per paw 

In [16]:
dict_paws = pickle.load(open(video_name + "dict_paws.pickle",'rb'))
step_analytics_video(dict_paws, wn = 400)